This notebook has been prepared to show a basic workflow for DEA uses

# 1. Import libraries

## python libraries

In [1]:
import datacube
from datacube.utils import geometry
from datacube.storage import masking


import xarray as xr
import pandas as pd
import fiona

%matplotlib inline

## DEA scripts

In [2]:
# Import the custom script from the dea-notebooks repository.
import sys
import os.path
sys.path.append(os.path.expanduser('10_Scripts'))
import DEAPlotting
import DEADataHandling

# 2. Link to Datacube

In [3]:
dc = datacube.Datacube()

In [4]:
dc.list_products()

,name,description,gqa_final_gcp_count,platform,gqa_iterative_mean_xy,gqa_cep90,gsi,product_type,lon,gqa_error_message,...,gqa,sat_row,ancillary_quality,creation_time,orbit,gqa_mean_xy,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,,,,,,,
36,bom_rainfall_grids,Interpolated Rain Gauge Precipitation 1-Day Au...,NaN,BoM,NaN,NaN,NaN,rainfall,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:4326,"(-0.05, 0.05)",None,"(latitude, longitude)"
88,dsm,"dsm ancillary data for wofs confidence, 25 met...",NaN,None,NaN,NaN,NaN,dsm,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
32,dsm1sv10,DSM 1sec Version 1.0,NaN,SRTM,NaN,NaN,NaN,DEM,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:4326,"(-0.00027777777778, 0.00027777777778)",None,"(latitude, longitude)"
99,fc_percentile_albers_annual,"Landsat Fractional Cover percentile 25 metre, ...",NaN,"LANDSAT_5,LANDSAT_7,LANDSAT_8",NaN,NaN,NaN,fractional_cover_statistical_summary,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)","(100000.0, 100000.0)","(y, x)"
53,gamma_ray,The 2015 radiometric or gamma-ray grid of Aust...,NaN,aircraft,NaN,NaN,NaN,gamma_ray,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,"GEOGCS[""GEOCENTRIC DATUM of AUSTRALIA"",DATUM[""...","(-0.001, 0.001)",None,"(latitude, longitude)"
63,high_tide_comp_20p,High Tide 20 percentage composites for entire ...,NaN,None,NaN,NaN,NaN,tidal_composite,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
64,high_tide_comp_count,High Tide 20 percentage pixel count,NaN,None,NaN,NaN,NaN,tidal_composite,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
67,item_v2,Intertidal Extents Model,NaN,None,NaN,NaN,NaN,ITEM,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"
68,item_v2_conf,Average ndwi Standard Deviation,NaN,None,NaN,NaN,NaN,ITEM,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,EPSG:3577,"(-25, 25)",None,"(y, x)"


# 3. Create query

In [ ]:
# Define temporal and spatial extents to pass to `dc.load` for the data extraction:
query = {'lat': (-35.25, -35.35),
         'lon': (149.05, 149.17),
         'time':('2017-01-01', '2017-06-30')}

or

In [5]:
# Set this to the location you of your polygon
shape_file = os.path.expanduser('Supplementary_data/Files/ACT_Central.shp')


In [10]:
with fiona.open(shape_file) as shapes:
    crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
    print(crs)
    for shape in shapes:
        if shape['geometry'] is None:
            continue
        geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
#         geom = geom.to_crs(CRS('EPSG:3577'))

GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]]


In [7]:
#create query        
query= {'geopolygon': geom,
         'time': ('2017-01-01', '2017-06-30')}
print(query)

NameError: name 'geom' is not defined

# 4. Load datacube data

In [ ]:
help(DEADataHandling.load_clearlandsat)

In [ ]:
ds = DEADataHandling.load_clearlandsat(dc=dc, query=query, sensors=('ls5', 'ls7', 'ls8'),
                                       product='nbart', mask_pixel_quality=False, mask_invalid_data=True,
#                                        bands_of_interest=['red', 'green', 'blue', 'nir', 'swir1'], 
                                       masked_prop=0.90)  

In [ ]:
ds

Now... Let's try:

1) Masking cloud

2)  fractional cover

# 5. Basic plots

In [ ]:
help(DEAPlotting.rgb)

In [ ]:
DEAPlotting.rgb(ds, bands=['red', 'green', 'blue'], index_dim = 'time', index=0,)
#DEAPlotting.rgb(ds, bands=['swir1',  'nir', 'green'], index_dim = 'time', index=0,)

In [ ]:
DEAPlotting.rgb(ds, bands=['swir1',  'nir', 'green'], index_dim = 'time',col="time", col_wrap=4)

In [ ]:
# ds.red.isel(time=0).plot(figsize=(10, 10));

# 6. Index calculation

Bands from Landsat surface reflectance products can be combined to compute simple remote sensing indices that provide insights into different aspects of the observed landscape. For example, we can calculate the Normalized Difference Water Index (NDWI), which has high values where a pixel is likely to be open water (e.g. NDWI > 0, or red colours below):

In [ ]:
# Compute NDWI using the formula (green - nir) / (green + nir). This will calculate
# NDWI for every time-step in the dataset:
ds_ndwi = (ds.green - ds.nir) / (ds.green + ds.nir)
ds_ndwi

In [ ]:
ds["NDWI"] = ds_ndwi
ds

In [ ]:
# First, we select the variable we want ('NDWI'), and then make a
# plot with one column for each time-step:
ndwi_image = ds.NDWI.isel(time=0).plot(figsize=(10, 10));

In [ ]:
ds.NDWI.plot(row='time', figsize=(5, 15));

In [ ]:
# Compute NDVI using the formula (nir - red) / (nir + red)
ds_ndvi = (ds.nir - ds.red) / (ds.nir + ds.red)

# Add this back into our dataset as a new data variable:
ds["NDVI"] = ds_ndvi

# Plot:
ds.NDVI.isel(time=0).plot(figsize=(10, 10));

In [ ]:
ds_ndvi